In [1]:
import pandas as pd

In [18]:
dataset = pd.read_json('News_Category_Dataset_v2.json', lines=True)

In [24]:
dataset = dataset.drop(['date', 'link', 'authors'], axis=1)

In [25]:
dataset.head(10)

,category,headline,short_description
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,She left her husband. He killed their children...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Of course it has a song.
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,The actor and his longtime girlfriend Anna Ebe...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,The actor gives Dems an ass-kicking for not fi...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,"The ""Dietland"" actress said using the bags is ..."
5,ENTERTAINMENT,Morgan Freeman 'Devastated' That Sexual Harass...,"""It is not right to equate horrific incidents ..."
6,ENTERTAINMENT,Donald Trump Is Lovin' New McDonald's Jingle I...,"It's catchy, all right."
7,ENTERTAINMENT,What To Watch On Amazon Prime That’s New This ...,There's a great mini-series joining this week.
8,ENTERTAINMENT,Mike Myers Reveals He'd 'Like To' Do A Fourth ...,"Myer's kids may be pushing for a new ""Powers"" ..."
9,ENTERTAINMENT,What To Watch On Hulu That’s New This Week,You're getting a recent Academy Award-winning ...


In [26]:
dataset['category'].unique()

array(['CRIME', 'ENTERTAINMENT', 'WORLD NEWS', 'IMPACT', 'POLITICS',
       'WEIRD NEWS', 'BLACK VOICES', 'WOMEN', 'COMEDY', 'QUEER VOICES',
       'SPORTS', 'BUSINESS', 'TRAVEL', 'MEDIA', 'TECH', 'RELIGION',
       'SCIENCE', 'LATINO VOICES', 'EDUCATION', 'COLLEGE', 'PARENTS',
       'ARTS & CULTURE', 'STYLE', 'GREEN', 'TASTE', 'HEALTHY LIVING',
       'THE WORLDPOST', 'GOOD NEWS', 'WORLDPOST', 'FIFTY', 'ARTS',
       'WELLNESS', 'PARENTING', 'HOME & LIVING', 'STYLE & BEAUTY',
       'DIVORCE', 'WEDDINGS', 'FOOD & DRINK', 'MONEY', 'ENVIRONMENT',
       'CULTURE & ARTS'], dtype=object)

In [35]:
target = (dataset['category'].map(lambda x: "WORLDPOST" if x == "THE WORLDPOST" else x)).tolist()

In [36]:
target[0]

'CRIME'

In [33]:
texts = (dataset['headline'] + ' ' + dataset['short_description']).tolist()

In [34]:
texts[0]

'There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV She left her husband. He killed their children. Just another day in America.'